# Import Libraries/Packages & Datasets

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import time
import copy

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

torch.backends.cudnn.benchmark=True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.manual_seed(5703)
torch.manual_seed(5703)
np.random.seed(5703)
random.seed(5703)

## Set Working directory
## Load benign data, drop unwanted columns
 

In [2]:
fulldata = pd.read_csv('idsnew\CICIDS_ALLATTACKS.csv')
fulldata['Label'].value_counts()

Benign                      4836398
DDoS attacks-LOIC-HTTP       573347
DoS attacks-Hulk             439126
DDOS attack-HOIC             360833
Bot                          285763
FTP-BruteForce               193354
SSH-Bruteforce               187589
Infilteration                152874
DoS attacks-SlowHTTPTest     139890
DoS attacks-GoldenEye         39924
DoS attacks-Slowloris          2724
DDOS attack-LOIC-UDP           1730
Brute Force -Web & XSS          544
Name: Label, dtype: int64

In [3]:
train_datas = {}
test_datas = {}
train_datas_soft = {}
train_datas_soft_no_benign = {}
sel_labels = []
mixed_data = pd.DataFrame()
for i in fulldata['Label'].unique():
    if fulldata[fulldata['Label'] == i].shape[0] > 100000:
        print(i)
        sel_labels.append(i)
        train_datas[i] = fulldata[fulldata['Label'] == i][:100000]
        test_datas[i] = fulldata[fulldata['Label'] == i][100000:110000]
        mixed_data = pd.concat([mixed_data, fulldata[fulldata['Label'] == i][110000:120000]], axis=0)
        train_datas_soft_no_benign[i] = fulldata[fulldata['Label'] == i][:100000]
        if i != 'Benign':
            train_datas_soft[i] = pd.concat([fulldata[fulldata['Label'] == 'Benign'][50000:100000], fulldata[fulldata['Label'] == i][50000:100000]], axis=0)

Benign
Bot
DDOS attack-HOIC
DDoS attacks-LOIC-HTTP
DoS attacks-Hulk
DoS attacks-SlowHTTPTest
FTP-BruteForce
Infilteration
SSH-Bruteforce


In [4]:
from sklearn.preprocessing import OneHotEncoder
labelencoder = OneHotEncoder()
labels = np.array(sel_labels).reshape(-1,1)
labelencoder.fit(labels)
encodedlabels = (labelencoder.transform(labels).todense()).tolist()
print(encodedlabels)
dikt = {}
for i in range(len(encodedlabels)):
    dikt[labels[i][0]] = encodedlabels[i]
dikt

[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]


{'Benign': [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'Bot': [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'DDOS attack-HOIC': [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'DDoS attacks-LOIC-HTTP': [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'DoS attacks-Hulk': [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 'DoS attacks-SlowHTTPTest': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 'FTP-BruteForce': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 'Infilteration': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 'SSH-Bruteforce': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]}

In [5]:
"""for i,row in soft_data.iterrows():
    soft_data.at[i,'Label'] = dikt[soft_data.at[i,'Label']]"""

"for i,row in soft_data.iterrows():\n    soft_data.at[i,'Label'] = dikt[soft_data.at[i,'Label']]"

In [6]:
del fulldata

In [7]:
mixed_data = mixed_data.sample(frac=1).reset_index(drop=True)
mixed_data_labels = list(mixed_data['Label'])
mixed_data.drop(['Label'], axis=1, inplace=True)
mixed_data = mixed_data.to_numpy()
for i in train_datas:
    train_datas[i].drop(['Label'], axis=1, inplace=True)
    test_datas[i].drop(['Label'], axis=1, inplace=True)
    train_datas[i] = train_datas[i].sample(frac=1).reset_index(drop=True)
    test_datas[i] = test_datas[i].sample(frac=1).reset_index(drop=True)
    train_datas[i] = train_datas[i].to_numpy()
    test_datas[i] = test_datas[i].to_numpy()

In [8]:
train_datas_soft_labels = {}
for i in train_datas_soft:
    train_datas_soft[i] = train_datas_soft[i].sample(frac=1).reset_index(drop=True)
    train_datas_soft_labels[i] = list(train_datas_soft[i]['Label'])
    train_datas_soft[i].drop(['Label'], axis=1, inplace=True)
    train_datas_soft[i] = train_datas_soft[i].to_numpy()

In [ ]:
train_datas_soft_no_benign_labels = {}
for i in train_datas_soft_no_benign:
    train_datas_soft_no_benign[i] = train_datas_soft_no_benign[i].sample(frac=1).reset_index(drop=True)
    train_datas_soft_no_benign_labels[i] = list(train_datas_soft_no_benign[i]['Label'])
    train_datas_soft_no_benign[i].drop(['Label'], axis=1, inplace=True)
    train_datas_soft_no_benign[i] = train_datas_soft_no_benign[i].to_numpy()

In [9]:
train_datas_soft_labels_encoded = {}
for i in train_datas_soft_labels:
    train_datas_soft_labels_encoded[i] = []
    for j in train_datas_soft_labels[i]:
        train_datas_soft_labels_encoded[i].append(dikt[j])

In [ ]:
train_datas_soft_no_benign_labels_encoded = {}
for i in train_datas_soft_no_benign_labels:
    train_datas_soft_no_benign_labels_encoded[i] = []
    for j in train_datas_soft_no_benign_labels[i]:
        train_datas_soft_no_benign_labels_encoded[i].append(dikt[j])

In [10]:
mixed_data_labels_encoded = []
for i in mixed_data_labels:
    mixed_data_labels_encoded.append(dikt[i])

In [11]:
#loading data into pytorch dataloader as train and test
devices = []
batch_size = 128
train_loader = {}
for i in train_datas:
  train_loader[i] = torch.utils.data.DataLoader(train_datas[i], batch_size = batch_size, shuffle=True)
  devices.append(i)

In [12]:
train_soft_loader = {}
for i in train_datas_soft:
  train_soft_loader[i] = torch.utils.data.DataLoader(train_datas_soft[i], batch_size = batch_size)

In [ ]:
train_soft_no_benign_loader = {}
for i in train_datas_soft_no_benign:
  train_soft_no_benign_loader[i] = torch.utils.data.DataLoader(train_datas_soft_no_benign[i], batch_size = batch_size)

In [13]:
len(train_loader)

9

In [14]:
len(train_soft_loader)

8

In [15]:
len(devices)

9

In [16]:
#initialising config variables
num_clients = len(train_loader)      # Number of clients
num_selected = len(train_loader)    # Typically, num_selected is around 30–40% of the num_clients.
#baseline_num = 1000  # choose some data from the train set to retrain the data from trained model
num_rounds = 2  #100      # Total number of communication rounds for the global model to train.
epochs = 10          # for train client model
#retrain_epochs = 3  # Total number of retraining rounds on the global server after receiving the model weights
                      # from all the clients that participated in the communication round.

## FedAvg / FedAvgM

In [17]:
# aggregates the model weights received from every client
# and updates the global model with updated weights

# FedAvg
def server_aggregate(global_model, client_models, client_lens):
    total = sum(client_lens)
    n = len(client_models)
    # n = num_selected
    global_dict = global_model.state_dict()
    for k in global_dict.keys(): # calculate average weight/bias --> avg_w/b
        global_dict[k] -= torch.stack([client_models[i].state_dict()[k].float() * (n * client_lens[i] / total) for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    #for model in client_models:
     #   model.load_state_dict(global_model.state_dict()) # local model get updated weight/bias

# FedAvgM
def server_aggregate_M(global_model, client_models, client_lens):
    total = sum(client_lens)    # 592    sum [51, 122, 162, 257]
    n = len(client_models)      # 4 local clients
    global_dict = global_model.state_dict() # weight/bias dict --> {'encoder.0.weight': Tensor with shape torch.Size([86, 115]), 'encoder.0.bias':....} 16 items
    temp = copy.deepcopy(global_dict)       # temporary weight/bias dict
    v = {x:1 for x in copy.deepcopy(global_dict)}   # initialise v

    for i,k in enumerate(global_dict.keys()):
        # calculate average weight/bias --> avg_w/b
        temp[k] = torch.stack([client_models[i].state_dict()[k].float() * (n * client_lens[i] / total) for i in range(len(client_models))], 0).mean(0)
        temp_v = 0.9 * v[k] + temp[k]               # v = 0.9v + avg_w/b   momentum=0.9
        global_dict[k] = global_dict[k] - temp_v    # w = w - v
    global_model.load_state_dict(global_dict)

    #for model in client_models:
     #   model.load_state_dict(global_model.state_dict()) # local model get updated weight/bias



In [18]:
# for updates/trains client model on client data
# local training round that takes place for every selected client

def client_update(client_model, optimizer, train_data, epoch=3):
    client_model.train()
    for e in range(epoch):
        running_loss = 0.0
        for data in train_data:
          output = client_model(data.float()) # tensor 115
          optimizer.zero_grad()
          # criterion = nn.MSELoss(reduction='mean')
          # loss = criterion(data.to(device), output)
          loss = nn.MSELoss(reduction='mean')(data.float().to(device), output)
          loss.backward()
          optimizer.step()
          # print(loss.item())
          running_loss += loss.item()
        # print(running_loss)
        epoch_loss = running_loss/len(train_data)
        # metrics['train_loss'].append(epoch_loss)
    return epoch_loss
    # return loss.item()



# synchronizes the client model with global weights (before training)

def client_syn(client_model, global_model):
    client_model.load_state_dict(global_model.state_dict())

## Deep Auto-encoder Model

In [19]:
import torch.nn as nn

input_dim = train_loader['Benign'].dataset.shape[1]

class AEModel(nn.Module):
    def __init__(self, input_dim):
        super(AEModel, self).__init__()
        
        xavier_gain = nn.init.calculate_gain('tanh')
        
        self.fc1 = nn.Linear(input_dim, input_dim)
        self.fc2 = nn.Linear(input_dim, 32)  
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 8)
        self.fc6 = nn.Linear(8, 16)
        self.fc7 = nn.Linear(16, 32)
        self.fc8 = nn.Linear(32, input_dim)
        self.fc9 = nn.Linear(input_dim, input_dim)
       
        self.activation = nn.Tanh()
        
        nn.init.xavier_uniform_(self.fc1.weight, gain=xavier_gain)
        nn.init.xavier_uniform_(self.fc2.weight, gain=xavier_gain)
        nn.init.xavier_uniform_(self.fc3.weight, gain=xavier_gain) 
        nn.init.xavier_uniform_(self.fc4.weight, gain=xavier_gain)
        nn.init.xavier_uniform_(self.fc5.weight, gain=xavier_gain)
        nn.init.xavier_uniform_(self.fc6.weight, gain=xavier_gain)
        nn.init.xavier_uniform_(self.fc7.weight, gain=xavier_gain)
        nn.init.xavier_uniform_(self.fc8.weight, gain=xavier_gain)
        nn.init.xavier_uniform_(self.fc9.weight, gain=xavier_gain)
        
    def forward(self, x):
        x = self.fc1(x) 
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x) 
        x = self.activation(x)
        x = self.fc5(x)
        x = self.activation(x) 
        x = self.fc6(x)
        x = self.activation(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.fc8(x) 
        x = self.activation(x)
        x = self.fc9(x)
        return x

## Initialisation

In [20]:
# Initializing models and optimizer

global_model = AEModel(input_dim).to(device)
client_models = [AEModel(input_dim).to(device) for _ in range(num_selected)] # part or all clients
# print(client_models)
for model in client_models:
    model.load_state_dict(global_model.state_dict())

# method_env = {'lr': (0.012, 0.012, 0.012, 0.0005)}
#opt = [torch.optim.SGD(model.parameters(), lr = 0.012, weight_decay=1e-05, momentum=0.9) for model in client_models]
# lr_list = [0.012, 0.012, 0.012, 0.00005]
# # opt = [torch.optim.SGD(model.parameters(), lr = lr_list[client_models.index(model)], weight_decay=1e-05, momentum=0.9) for model in client_models]
# opt = [torch.optim.Adam(model.parameters(), lr = lr_list[client_models.index(model)], weight_decay=1e-05) for model in client_models]
opt = [torch.optim.Adam(model.parameters(), lr = 0.012, weight_decay=1e-05) for model in client_models]

# Training Federated Model

In [ ]:
# Train Client Model and Global Model
train_loss_client = []
train_loss_global = []
train_loss_per_client = [[] for i in range(num_clients)]
train_loss_benign = []

start_time = time.time()

for r in range(num_rounds): # total number of rounds

    print('\nround: ', r+1)
    # num_selected = random.choice(range(1,num_clients))
    #client_idx = np.random.permutation(num_clients)[:num_selected] # random pick some clients from all clients for train
    client_idx = list(range(num_clients))
    print('client_idx: ', client_idx)
    client_lens = [len(train_loader[devices[idx]]) for idx in client_idx ] # Need to fix this. It is part of server aggregation.
    # client_lens = [len(train_lotrain_loaderader[idx]) for idx in client_idx ]
    #print(client_lens)


    #### client update ####
    loss = 0
    for i in tqdm(range(num_selected)):
        # print(i)
        #client_syn(client_models[i], global_model)
        # print(train_loader[devices[client_idx[i]]])
        if devices[client_idx[i]] != 'Benign':
            l = client_update(client_models[i], opt[i], train_loader[devices[client_idx[i]]], epochs)
            train_loss_per_client[i].append(l)
            loss += l
    train_loss_client.append(loss)

    #### benign client update ####
    for i in tqdm(range(1)):
        l = client_update(client_models[-1], opt[-1], train_loader['Benign'], epochs)
        train_loss_benign.append(l)
    train_loss_per_client[-1] = train_loss_benign
    train_loss_client.append(sum(train_loss_benign))


    #### retraining on the global server ####
    #loss_retrain = 0
    #for i in tqdm(range(num_selected)):
    #    loss_retrain += client_update(client_models[i], opt[i], baseline_data[devices[client_idx[i]]], retrain_epochs)
    #train_loss_global.append(loss_retrain/num_selected)

    attackclients = []
    for i in devices:
        if i != 'Benign':
            attackclients.append(client_models[devices.index(i)])
    
    ### aggregate models ###
    server_aggregate(global_model, attackclients, client_lens)      # FedAvg
    #server_aggregate_M(global_model, attackclients, client_lens)    # FedAvgM

    print("\nclient_loss: ", loss)
    #print('global_loss: ', loss_retrain/num_selected)

time_required = time.time() - start_time
print('/nTIME: {}mins'.format(time_required/60))

In [ ]:
for i in range(len(train_loss_per_client)):
  print(train_loss_per_client[i])

In [ ]:
# save model
torch.save(global_model,'C:/Final Year Project/modelsnew/globalattackmodel.pt')

In [ ]:
for i in range(len(devices)):
  torch.save(client_models[i],'C:/Final Year Project/modelsnew/clients/' + devices[i] + '.pt')

In [ ]:
losspic = 1
def printperformance(losses, modelname):
    global losspic
    plt.plot(losses)
    plt.xlabel('number of rounds')
    plt.ylabel('loss')
    plt.ylim((0, 2.5))
    plt.grid()
    plt.title(modelname)
    plt.show()
    plt.savefig("C:/Final Year Project/images/" + str(modelname) + str(losspic) + ".png")
    losspic += 1

In [ ]:
for i in range(len(devices)-1):
  printperformance(train_loss_per_client[i], devices[i])

In [ ]:
# print performance of benign client
printperformance(train_loss_benign, devices[-1])

In [ ]:
# train_loss_global
plt.plot(train_loss_global)
plt.xlabel('number of rounds')
plt.ylabel('loss')
plt.ylim((0, 2.5))
plt.grid()
plt.title('Federated Model with FedAvg (use five devices to train)')
plt.show()
plt.savefig("C:/Final Year Project/images/Global10.png")

# Evaluation

In [21]:
client_models = {}
for device in devices:
  client_models[device] = torch.load('modelsnew/clients/' + device + '.pt')
global_model = torch.load('modelsnew/globalattackmodel.pt')

In [22]:
def forward(self, x):
        x = self.fc1(x) 
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x) 
        x = self.activation(x)
        x = self.fc5(x)
        x = self.activation(x) 
        x = self.fc6(x)
        x = self.activation(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.fc8(x) 
        x = self.activation(x)
        x = self.fc9(x)
        x = self.activation(x)
        #x = x.reshape((1, -1))
        x = self.fc10(x)
        #x = self.Softmax(x)
        return x

In [23]:
for i in client_models:
    client_models[i].add_module('fc10', nn.Linear(input_dim, len(devices)))
    #client_models[i].add_module('Softmax', nn.Softmax(dim=1))
    nn.init.xavier_uniform_(client_models[i].fc10.weight, gain=nn.init.calculate_gain('tanh'))
    client_models[i].__class__.forward = forward
global_model.add_module('fc10', nn.Linear(input_dim, len(devices)))
#global_model.add_module('Softmax', nn.Softmax(dim=1))
nn.init.xavier_uniform_(global_model.fc10.weight, gain=nn.init.calculate_gain('tanh'))
global_model.__class__.forward = forward

In [24]:
def client_update_soft(client_model, optimizer, train_data, target_labels, epoch=3):
    client_model.train()
    for e in range(epoch):
        running_loss = 0.0
        prev = 0
        ind = 128
        for data in train_data:
          output = client_model(data.float()) # tensor 115
          optimizer.zero_grad()
          # criterion = nn.MSELoss(reduction='mean')
          # loss = criterion(data.to(device), output)
          target_label = torch.Tensor(target_labels[prev:ind])
          loss = nn.CrossEntropyLoss()(output, target_label)
          loss.backward()
          optimizer.step()
          # print(loss.item())
          running_loss += loss.item()
          prev = ind
          ind += 128
          if ind > len(target_labels):
            ind = len(target_labels)
        # print(running_loss)
        epoch_loss = running_loss/len(train_data)
        # metrics['train_loss'].append(epoch_loss)
    return epoch_loss

In [25]:
# Train Client Model and Global Model
train_loss_client = []
train_loss_global = []
train_loss_per_client = [[] for i in range(num_clients)]
train_loss_benign = []

start_time = time.time()

for r in range(num_rounds): # total number of rounds

    print('\nround: ', r+1)
    # num_selected = random.choice(range(1,num_clients))
    #client_idx = np.random.permutation(num_clients)[:num_selected] # random pick some clients from all clients for train
    client_idx = list(range(num_clients))
    print('client_idx: ', client_idx)
    client_lens = [len(train_loader[devices[idx]]) for idx in client_idx ] # Need to fix this. It is part of server aggregation.
    # client_lens = [len(train_lotrain_loaderader[idx]) for idx in client_idx ]
    #print(client_lens)


    #### client update ####
    loss = 0
    for i in tqdm(range(num_selected)):
        # print(i)
        #client_syn(client_models[i], global_model)
        # print(train_loader[devices[client_idx[i]]])
        if devices[client_idx[i]] != 'Benign':
            model = client_models[devices[client_idx[i]]]
            l = client_update_soft(client_models[devices[client_idx[i]]], opt[i], train_soft_loader[devices[client_idx[i]]], train_datas_soft_labels_encoded[devices[client_idx[i]]], epochs)
            train_loss_per_client[i].append(l)
            loss += l
    train_loss_client.append(loss)

    #### benign client update ####
    #for i in tqdm(range(1)):
    #    l = client_update_soft(client_models['Benign'], opt[-1], train_loader['Benign'], dikt['Benign'], epochs)
    #    train_loss_benign.append(l)
    #train_loss_per_client[-1] = train_loss_benign
    #train_loss_client.append(sum(train_loss_benign))


    #### retraining on the global server ####
    #loss_retrain = 0
    #for i in tqdm(range(num_selected)):
    #    loss_retrain += client_update(client_models[i], opt[i], baseline_data[devices[client_idx[i]]], retrain_epochs)
    #train_loss_global.append(loss_retrain/num_selected)

    attackclients = []
    for i in devices:
        if i != 'Benign':
            attackclients.append(client_models[i])
    
    ### aggregate models ###
    server_aggregate(global_model, attackclients, client_lens)      # FedAvg
    #server_aggregate_M(global_model, attackclients, client_lens)    # FedAvgM

    print("\nclient_loss: ", loss)
    #print('global_loss: ', loss_retrain/num_selected)

time_required = time.time() - start_time
print('/nTIME: {}mins'.format(time_required/60))


round:  1
client_idx:  [0, 1, 2, 3, 4, 5, 6, 7, 8]


100%|██████████| 9/9 [07:28<00:00, 49.85s/it]



client_loss:  18.924817600213657

round:  2
client_idx:  [0, 1, 2, 3, 4, 5, 6, 7, 8]


100%|██████████| 9/9 [07:44<00:00, 51.62s/it]


client_loss:  18.924817600213657
/nTIME: 15.224013968308766mins


In [27]:
# save model
torch.save(global_model,'modelssoft/globalattackmodel.pt')
for i in client_models.keys():
  torch.save(client_models[i],'modelssoft/clients/' + i + '.pt')

In [29]:
client_models = {}
for device in devices:
  client_models[device] = torch.load('modelssoft/clients/' + device + '.pt')
global_model = torch.load('modelssoft/globalattackmodel.pt')

In [30]:
dikt

{'Benign': [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'Bot': [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'DDOS attack-HOIC': [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'DDoS attacks-LOIC-HTTP': [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'DoS attacks-Hulk': [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 'DoS attacks-SlowHTTPTest': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 'FTP-BruteForce': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 'Infilteration': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 'SSH-Bruteforce': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]}

In [31]:
from collections import Counter
device_idx = {device: i for i, device in enumerate(devices)}
res={}
for device in devices:
    temp = []
    model = client_models[device]
    model.eval()
    for data in mixed_data:
        data = torch.Tensor(data).reshape(1,-1)
        pred = nn.Softmax(dim=1)(model(data).float())
        pred = pred.tolist()[0]
        temp.append(pred.index(max(pred)))
    res[device] = Counter(temp).most_common(1)[0][0]
print(res)
            

{'Benign': 0, 'Bot': 7, 'DDOS attack-HOIC': 7, 'DDoS attacks-LOIC-HTTP': 4, 'DoS attacks-Hulk': 2, 'DoS attacks-SlowHTTPTest': 4, 'FTP-BruteForce': 1, 'Infilteration': 8, 'SSH-Bruteforce': 4}


In [32]:
model = global_model
model.eval()
labels = mixed_data_labels
res_global=[]
print(mixed_data.shape)
for i in range(mixed_data.shape[0]):
    #if mixed_data_labels[i]=="SSH-Bruteforce":
    data = torch.Tensor(mixed_data[i]).reshape(1,-1)
    pred = nn.Softmax(dim=1)(model(data).float())
    pred = pred.tolist()[0]
    res_global.append(pred.index(max(pred)))

(90000, 69)


In [33]:
from collections import Counter
r = Counter(res_global)
r

Counter({2: 89941, 4: 56, 1: 3})

In [37]:
device_idx = {device: i for i, device in enumerate(devices)}
temp_test = []
for device in devices:
    if device =="FTP-BruteForce":
        model = client_models[device]
        model.eval()
        for data in mixed_data:
            data = torch.Tensor(data).reshape(1,-1)
            pred = nn.Softmax(dim=1)(model(data).float())
            pred = pred.argmax(dim=1)
            temp_test.append(pred)

In [38]:
temp_test

[tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tensor([1]),
 tenso

In [34]:
mixed_data_labels

['DoS attacks-SlowHTTPTest',
 'FTP-BruteForce',
 'Benign',
 'SSH-Bruteforce',
 'FTP-BruteForce',
 'Bot',
 'SSH-Bruteforce',
 'FTP-BruteForce',
 'Benign',
 'DoS attacks-Hulk',
 'FTP-BruteForce',
 'DDoS attacks-LOIC-HTTP',
 'DoS attacks-Hulk',
 'Benign',
 'DDoS attacks-LOIC-HTTP',
 'DoS attacks-Hulk',
 'DDoS attacks-LOIC-HTTP',
 'SSH-Bruteforce',
 'DoS attacks-SlowHTTPTest',
 'SSH-Bruteforce',
 'Infilteration',
 'Benign',
 'DoS attacks-SlowHTTPTest',
 'DDoS attacks-LOIC-HTTP',
 'DoS attacks-SlowHTTPTest',
 'Benign',
 'Benign',
 'Infilteration',
 'Bot',
 'DDoS attacks-LOIC-HTTP',
 'Infilteration',
 'Infilteration',
 'Benign',
 'Bot',
 'DoS attacks-SlowHTTPTest',
 'DoS attacks-SlowHTTPTest',
 'Infilteration',
 'DDOS attack-HOIC',
 'Benign',
 'Infilteration',
 'Benign',
 'DoS attacks-Hulk',
 'DDoS attacks-LOIC-HTTP',
 'DDoS attacks-LOIC-HTTP',
 'DDoS attacks-LOIC-HTTP',
 'Infilteration',
 'DoS attacks-SlowHTTPTest',
 'Benign',
 'Benign',
 'DDOS attack-HOIC',
 'Benign',
 'DoS attacks-SlowHTT